In [1]:
import pandas as pd
from pathlib import Path
from astropy.time import Time
from mirar.data import SourceBatch, SourceTable, Dataset
from mirar.pipelines import get_pipeline
from mirar.database.constraints import DBQueryConstraints
from mirar.database.transactions.update import _update_database_entry
from mirar.pipelines.winter.models import Candidate
import logging
import numpy as np
from tqdm import tqdm

In [2]:
pd.options.mode.copy_on_write = True

In [3]:
# logging.basicConfig(level=logging.INFO)

In [4]:
path = Path.home().joinpath("Downloads/candidate_names_20240220.csv")
path = Path.home().joinpath("export_candidates.csv")
print(f"Path is {path}")
assert path.exists, "Failed to find path"

Path is /Users/robertstein/export_candidates.csv


In [5]:
df = pd.read_csv(path)
df.drop(["objectid", "candid", "sourceid"], axis=1, inplace=True)

In [6]:
df.sort_values(by="jd", inplace=True)
df

,deprecated,jd,diffid,stackid,fid,exptime,progname,isdiffpos,fieldid,ra,...,tmkmag2,tmobjectid2,tmjmag3,tmhmag3,tmkmag3,tmobjectid3,neargaia,neargaiabright,maggaia,maggaiabright
282,f,2.460113e+06,3,9954683705,2,600,2001A000,t,3944,211.234943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,f,2.460153e+06,3,9954683703,2,600,2001A000,t,3944,211.036962,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10207,t,2.460153e+06,3,9954683703,2,600,2001A000,t,3944,210.719983,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10208,t,2.460153e+06,3,9954683703,2,600,2001A000,t,3944,210.678061,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10209,t,2.460153e+06,3,9954683703,2,600,2001A000,t,3944,210.648139,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6122,t,2.460249e+06,65,10784307506,2,900,2001A000,t,999999999,349.991030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1617,f,2.460249e+06,65,10784307506,2,900,2001A000,t,999999999,350.338046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6138,t,2.460249e+06,65,10784307506,2,900,2001A000,t,999999999,350.335105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6089,t,2.460249e+06,65,10784307506,2,900,2001A000,t,999999999,350.319281,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
for jd in tqdm(sorted(set(df["jd"]))):
    pipe = get_pipeline("WINTER", selected_configurations="name_candidates")

    dataset = Dataset()
    
    df_sub = df[df["jd"] == jd]
    df_sub.replace({np.nan: None}, inplace=True)

    # Deprecate old candidates
    for stackid in set(df_sub["stackid"]):
        dbconstraints = DBQueryConstraints()
        dbconstraints.add_constraint(
            column="stackid",
            accepted_values=stackid,
        )
        _update_database_entry(
            update_dict={"deprecated": True},
            sql_table=Candidate.sql_model,
            db_constraints=dbconstraints,
        )

    metadata = {
        "RAWPATH": "history",
        "BASENAME": "history",
        "CALSTEPS": "",
        "DATE-OBS": Time(jd, format="jd").isot,
        "utctime": Time(jd, format="jd").isot,
        "diffmaglim": (df_sub["magpsf"] + df_sub["magfromlim"]).iloc[0],
        "jd": jd
    }
    df_sub["DATE-OBS"] = Time(jd, format="jd").isot
    # df_sub["diffmaglim"] = np.nan

    n_chunk = 10

    for df_chunk in [df_sub[i:i+n_chunk] for i in range(0,df_sub.shape[0],n_chunk)][:3]:

        df_chunk.reset_index(inplace=True)
        
        source_table = SourceTable(source_list=df_chunk, metadata=metadata)
        batch = SourceBatch()
        batch.append(source_table)
        dataset.append(batch)

    batches, errorstack = pipe.reduce_images(dataset=dataset, catch_all_errors=False, selected_configurations=["name_candidates"])
    # print(errorstack.summarise_error_stack(verbose=True))

  0%|                                                                                                        | 0/6 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Saving tracebacks of caught errors to /Users/robertstein/Data/winter/_error_stack.txt
Saving tracebacks of caught errors to /Users/robertstein/Data/winter/_error_stack.tsv
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:24<00:00,  4.05s/it]
